In [1]:
!pip install --upgrade xgboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
from tqdm.auto import tqdm
import numpy as np
from datetime import date, timedelta
import time
from google.colab import drive
drive.mount('/content/drive')
import random
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, PowerTransformer,QuantileTransformer ,LabelEncoder, OneHotEncoder, MinMaxScaler
import seaborn as sns
! pip install prince
import prince

! pip install optuna
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

     |████████████████████████████████| 157.5MB 67kB/s 
  Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90
Mounted at /content/drive
     |████████████████████████████████| 296kB 5.3MB/s 
     |████████████████████████████████| 1.2MB 33.4MB/s 
     |████████████████████████████████| 81kB 9.5MB/s 
     |████████████████████████████████| 481kB 44.6MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 143kB 34.5MB/s 
     |████████████████████████████████| 112kB 41.8MB/s 
  Created wheel for alembic: filename=alembic-1.5.5-py2.py3-none-any.whl size=156597 sha256=07fb002e4cc5140bfa96ff4cbea1713f91f1c566bfb5869d01e9840cfba3c492
  Stored in directory: /root/.cache/pip/wheels/74/3f/61/7de6e3cef766d2680a5d81b1a388286e640f6a681eb589d643
  Created wheel for Mako: filename=Mako-1.1.4-py2.py3-none-any.whl size=75675 sha256=c2f25ba3264e71b4040e587fa86c743e600cb3d4adfcc6cec7390a70e1ab4dc

In [2]:
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [3]:
path = '/content/drive/Shareddrives/dacon/Playground/'
train = pd.read_csv(path + 'train.csv', index_col=['id'])
test = pd.read_csv(path + 'test.csv', index_col=['id'])
submission = pd.read_csv(path + 'sample_submission.csv', index_col=['id'])

train_data = train.iloc[:,:-1]
train_target = train.iloc[:, -1]
test_data = test.copy()

In [4]:
cat_features = [feature for feature in train_data.columns if 'cat' in feature]
cont_features = [feature for feature in train_data.columns if 'cont' in feature]
for feature in cat_features:
    train_data[feature] = train_data[feature].astype('category')
    test_data[feature] = test_data[feature].astype('category')

In [5]:
def dropper(dataset):
  df = dataset.copy()
  df['cat3'] = df['cat3'].apply(lambda x: x if x in ['A', 'C'] else 'B')
  df['cat5'] = df['cat5'].apply(lambda x: x if x == 'C' else 'B')
  df['cat6'] = df['cat6'].apply(lambda x: x if x in ['A', 'B', 'C'] else 'D')
  df['cat7'] = df['cat7'].apply(lambda x: 'G' if x in ['G', 'E'] else x)
  df['cat7'] = df['cat7'].apply(lambda x: 'D' if x in ['B', 'D'] else x)
  df['cat8'] = df['cat8'].apply(lambda x: 'A' if x in ['A', 'D'] else x)
  df['cat9'] = df['cat9'].apply(lambda x: 'B' if x in ['O', 'B', 'K'] else x)
  df['cat9'] = df['cat9'].apply(lambda x: 'H' if x in ['N', 'H', 'M'] else x)
  return df

In [6]:
train_temp = train_data.copy()
test_temp = test_data.copy()

cat_features = [feature for feature in train_temp.columns if 'cat' in feature]
cont_features = [feature for feature in train_temp.columns if 'cont' in feature]
scaler  = QuantileTransformer(random_state=42, n_quantiles=1500)

train_data[cont_features] = scaler.fit_transform(train_data[cont_features])
test_data[cont_features] = scaler.transform(test_data[cont_features])

train_data[cat_features] = dropper(train_temp[cat_features])
test_data[cat_features] = dropper(test_data[cat_features])

train_data = pd.concat([train_data, train_target], axis = 1)

In [8]:
for cat in cat_features:
  le = LabelEncoder()
  le.fit(train_data[cat])
  train_data[cat] = le.transform(train_data[cat])
  test_data[cat] = le.transform(test_data[cat])

In [9]:
# Outlier 제거
trainset = train_data.copy()
trainset.drop(['target'], axis = 1, inplace = True)

from sklearn.ensemble import IsolationForest
iso_model = IsolationForest(max_samples='auto', random_state=42, n_jobs = -1, contamination = 0.05, n_estimators=500)
iso_model.fit(trainset)

score = iso_model.decision_function(trainset)
train_data['scores'] = score

tcore = iso_model.decision_function(test_data)
test_data['scores'] = tcore

In [24]:
def create_stratified_folds_for_regression(data_df, n_splits=4):
    """
    @param data_df: training data to split in Stratified K Folds for a continous target value
    @param n_splits: number of splits
    @return: the training data with a column with kfold id
    """
    data_df['kfold'] = -1
    # randomize the data
    data_df = data_df.sample(frac=1, random_state=42).reset_index(drop=True)
    # calculate the optimal number of bins based on log2(data_df.shape[0])
    num_bins = np.int(np.floor(1 + np.log2(len(data_df))))
    # bins value will be the equivalent of class value of target feature used by StratifiedKFold to 
    # distribute evenly the classed over each fold
    data_df.loc[:, "bins"] = pd.cut(pd.to_numeric(data_df['target'], downcast="signed"), bins=num_bins, labels=False)
    kf = StratifiedKFold(n_splits=n_splits, random_state=42)
    
    # set the fold id as a new column in the train data
    for f, (t_, v_) in enumerate(kf.split(X=data_df, y=data_df.bins.values)):
        data_df.loc[v_, 'kfold'] = f
    
    # drop the bins column (no longer needed)
    data_df = data_df.drop("bins", axis=1)
    
    return data_df

In [33]:
def objectiveXGB(trial: Trial, X):
    # param = {
    #     "n_estimators" : 500,
    #     'max_depth':trial.suggest_int('max_depth', 8, 16),
    #     'min_child_weight':trial.suggest_int('min_child_weight', 1, 300),
    #     'gamma':trial.suggest_int('gamma', 1, 3),
    #     'learning_rate': 0.01,
    #     'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.5, 1, 0.05),
    #     'nthread' : -1,
    #     'tree_method': 'gpu_hist',
    #     'predictor': 'gpu_predictor',
    #     'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
    #     'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
    #     'subsample': trial.suggest_discrete_uniform('subsample', 0.5,1, 0.05 ),
    #     'random_state': 42
    # }

    xgb_params = {}
    xgb_params['objective'] = 'reg:squarederror'
    xgb_params['n_estimators'] = 1000
    xgb_params['max_depth'] = trial.suggest_int('max_depth', 3, 16)
    xgb_params['min_child_weight'] = trial.suggest_int('min_child_weight', 1, 300)
    xgb_params['gamma'] = trial.suggest_discrete_uniform('gamma', 0.5, 10, 0.5)
    xgb_params['learning_rate'] = 0.01
    xgb_params['colsample_bytree'] = trial.suggest_discrete_uniform('colsample_bytree',0.5, 1, 0.05)
    xgb_params['tree_method'] = 'gpu_hist'
    xgb_params['gpu_id'] = 0
    xgb_params['lambda'] = trial.suggest_uniform('lambda', 1e-3, 100.0)
    xgb_params['alpha'] = trial.suggest_uniform('alpha', 1e-3, 100.0)
    xgb_params['subsample'] = trial.suggest_discrete_uniform('subsample', 0.5,1, 0.05 )
    xgb_params['random_state'] = 42

    model = XGBRegressor(**xgb_params)

    score = cv_evaluation_regression(X, model)

    return np.sqrt(score)

In [12]:
def cv_evaluation_regression(df, model):
    n_fold = 4
    oof = np.zeros((len(df),))
    for fold in tqdm(range(n_fold)):
        val_ind = df[df.kfold == fold].index
        train = df[df.kfold != fold].reset_index(drop=True)
        valid = df[df.kfold == fold].reset_index(drop=True)

        features = [feature for feature in df.columns if feature not in ['id', 'target', 'kfold']]

        X_train = train[features]
        y_train = train['target']
        X_valid = valid[features]
        y_valid = valid['target']

        model.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], early_stopping_rounds = 500, verbose=False)
        preds = model.predict(valid[features])
        mse = mean_squared_error(y_valid, preds)
        
        oof[val_ind] = preds

    mse_oof = mean_squared_error(oof, df['target'])
    return np.sqrt(mse_oof)

In [26]:
train_cv = create_stratified_folds_for_regression(train_data)

In [27]:
trainset = pd.concat([train_cv, train_target], axis = 1)

In [34]:
study = optuna.create_study(direction = 'minimize')
study.optimize(lambda trial : objectiveXGB(trial, train_cv), n_trials=200)

[I 2021-02-24 17:05:15,374] A new study created in memory with name: no-name-ad7b3314-89fa-4d8f-b753-b6e7db3d2eed


[I 2021-02-24 17:05:32,785] Trial 0 finished with value: 0.9220042920282583 and parameters: {'max_depth': 6, 'min_child_weight': 279, 'gamma': 4.0, 'colsample_bytree': 0.75, 'lambda': 55.13612225043196, 'alpha': 99.94938586274078, 'subsample': 0.75}. Best is trial 0 with value: 0.9220042920282583.


[I 2021-02-24 17:05:49,451] Trial 1 finished with value: 0.9219747090941198 and parameters: {'max_depth': 7, 'min_child_weight': 142, 'gamma': 8.0, 'colsample_bytree': 0.7, 'lambda': 8.569241547795373, 'alpha': 74.39032171367751, 'subsample': 0.65}. Best is trial 1 with value: 0.9219747090941198.


[I 2021-02-24 17:06:10,970] Trial 2 finished with value: 0.9210757403132169 and parameters: {'max_depth': 16, 'min_child_weight': 297, 'gamma': 6.0, 'colsample_bytree': 0.5, 'lambda': 31.20997858198537, 'alpha': 48.75972769073061, 'subsample': 0.7}. Best is trial 2 with value: 0.9210757403132169.


[I 2021-02-24 17:06:32,400] Trial 3 finished with value: 0.9205761713691709 and parameters: {'max_depth': 8, 'min_child_weight': 113, 'gamma': 8.0, 'colsample_bytree': 0.5, 'lambda': 63.485310104802785, 'alpha': 6.13621792274492, 'subsample': 0.8500000000000001}. Best is trial 3 with value: 0.9205761713691709.


...

[I 2021-02-24 21:24:17,367] Trial 195 finished with value: 0.9184566672793975 and parameters: {'max_depth': 14, 'min_child_weight': 121, 'gamma': 1.5, 'colsample_bytree': 0.5, 'lambda': 0.1111322680168505, 'alpha': 2.761903551476338, 'subsample': 0.8500000000000001}. Best is trial 184 with value: 0.9183515839479552.


[I 2021-02-24 21:26:00,884] Trial 196 finished with value: 0.9185519377369487 and parameters: {'max_depth': 14, 'min_child_weight': 106, 'gamma': 1.5, 'colsample_bytree': 0.5, 'lambda': 0.38916766365049327, 'alpha': 9.226619230458496, 'subsample': 0.8500000000000001}. Best is trial 184 with value: 0.9183515839479552.


[I 2021-02-24 21:27:50,051] Trial 197 finished with value: 0.9184627983807168 and parameters: {'max_depth': 14, 'min_child_weight': 116, 'gamma': 2.0, 'colsample_bytree': 0.5, 'lambda': 2.6127463087946987, 'alpha': 3.1808512806821057, 'subsample': 0.8500000000000001}. Best is trial 184 with value: 0.9183515839479552.


[I 2021-02-24 21:29:24,592] Trial 198 finished with value: 0.9185217512333759 and parameters: {'max_depth': 14, 'min_child_weight': 122, 'gamma': 2.0, 'colsample_bytree': 0.5, 'lambda': 2.330027771260118, 'alpha': 5.854849375388056, 'subsample': 0.8500000000000001}. Best is trial 184 with value: 0.9183515839479552.


[I 2021-02-24 21:31:14,125] Trial 199 finished with value: 0.9185378855390586 and parameters: {'max_depth': 14, 'min_child_weight': 114, 'gamma': 2.0, 'colsample_bytree': 0.5, 'lambda': 4.257607519977995, 'alpha': 3.0736042834341784, 'subsample': 0.8500000000000001}. Best is trial 184 with value: 0.9183515839479552.


In [35]:
optuna.visualization.plot_optimization_history(study)

In [36]:
study.best_params

{'alpha': 0.10778032483798261,
 'colsample_bytree': 0.5,
 'gamma': 1.5,
 'lambda': 1.4357867083946942,
 'max_depth': 14,
 'min_child_weight': 138,
 'subsample': 0.8}

In [37]:
study.optimize(lambda trial : objectiveXGB(trial, trainset, train_target), n_trials=100)

[W 2021-02-24 21:31:16,492] Trial 200 failed because of the following error: TypeError('objectiveXGB() takes 2 positional arguments but 3 were given')
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/optuna/_optimize.py", line 211, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-37-f992d943e443>", line 1, in <lambda>
    study.optimize(lambda trial : objectiveXGB(trial, trainset, train_target), n_trials=100)
TypeError: objectiveXGB() takes 2 positional arguments but 3 were given


TypeError: ignored

In [ ]:
study.best_params